In [39]:
import json
import pprint as pp
import numpy as np
import requests

In [15]:
with open("redlines_data.json") as f:
    data = json.load(f)
features = data["features"]
feature = features[0]
color = feature['properties'].get('holc_color')
print(color)

None


In [30]:
xgrid = np.arange(1,5,0.1)
ygrid = np.arange(2,5,0.1)
xmesh, ymesh = np.meshgrid(xgrid, ygrid)
np.vstack([xmesh.flatten(), ymesh.flatten()]).T

array([[1. , 2. ],
       [1.1, 2. ],
       [1.2, 2. ],
       ...,
       [4.7, 4.9],
       [4.8, 4.9],
       [4.9, 4.9]])

In [31]:
import pandas as pd
df = pd.read_json("redlines_cache.json")
df.head(5)

,coordinates,holcGrade,id,description,holcColor,randomLat,randomLong,medIncome,censusTract,rank,percent
0,"[[-83.13339, 42.438703], [-83.133276, 42.43817...",A,A1,60% developed. All city facilities. Highly res...,darkgreen,42.436,-83.124,0,26163538200,1,1
1,"[[-82.912275, 42.378742], [-82.914262, 42.3816...",A,A10,The neighborhood will probably hold up very we...,darkgreen,42.388,-82.904,0,26163550500,2,1
2,"[[-82.909919, 42.408288], [-82.910632, 42.4069...",A,A11,"This is a new section of modest houses, well l...",darkgreen,42.404,-82.908,0,26163550700,3,1
3,"[[-82.894626, 42.42049], [-82.883715, 42.41662...",A,A12,A high class residential neighborhood of good ...,darkgreen,42.400,-82.904,0,26163550700,4,1
4,"[[-82.89247399999999, 42.430993], [-82.891048,...",A,A13,This area is the location of many of the fines...,darkgreen,42.424,-82.884,0,26163550900,5,1


In [34]:
df.groupby('holcGrade')['rank'].mean()

holcGrade
A      8.0
B     35.0
C    115.5
D    207.5
Name: rank, dtype: float64

In [38]:
data = pd.read_csv("income_data.csv")

data['tract_id'] = data.apply(lambda row: f"{row['state']}{row['county']}{row['tract']}", axis=1)
data.head(5)

,B06011_001E,state,county,tract,tract_id
0,25114,26,137,950400,26137950400
1,26435,26,137,950500,26137950500
2,49485,26,125,150900,26125150900
3,60972,26,125,159000,26125159000
4,27315,26,59,50400,265950400


In [44]:
def fetch(params):
    """
    Fetches data from the U.S. Census Bureau's API.

    This method sends a GET request to the specified API endpoint with the provided parameters.
    If the request is successful, the method returns the JSON response. If the request fails,
    the method retries the request until it is successful.

    Parameters
    ----------
    params : dict
        A dictionary containing the parameters to be sent with the request.

    Returns
    -------
    dict
        A dictionary containing the JSON response from the API.

    """
    api = "https://api.census.gov/data/2018/acs/acs5"
    key = "bff2ebd794e77fac281f258fe892bd1e36fe481b"
    params["key"]=key
    #print(params)
    while True:
        response = requests.get(api, params)
        if response.status_code == 200:
            data = response.json()
            break
        else:
            print(response.status_code)
    return data


In [55]:
params = {"get": "B06011_001E", "for": "tract:*", "in": "state:26"}
data = fetch(params)

In [57]:
df = pd.DataFrame(data[1:], columns=data[0])
df.head(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2813 entries, 0 to 2812
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   B06011_001E  2813 non-null   object
 1   state        2813 non-null   object
 2   county       2813 non-null   object
 3   tract        2813 non-null   object
dtypes: object(4)
memory usage: 88.0+ KB


In [13]:
from xml.dom.expatbuilder import FILTER_ACCEPT
import numpy as np
import json
import os
import random
import matplotlib.pyplot as plt
from matplotlib.path import Path
from regex import F
import requests
import time
from tqdm import tqdm
import csv
import pandas as pd
import re
from collections import Counter
from nltk.corpus import stopwords
import string

random.seed(17)


class DetroitDistrict:
    """
    A class representing a district in Detroit with attributes related to historical redlining.
    coordinates,holcGrade,holcColor,id,description should be load from the redLine data file
    if cache is not available

    Parameters
    ------------------------------
    coordinates : list of lists, 2D List, not list of list of list
        Coordinates defining the district boundaries from the json file
        Note that some districts are non-contiguous, which may
        effect the structure of this attribute

    holcGrade : str
        The HOLC grade of the district.

    id : str
        The identifier for the district, the HOLC ID.

    description : str, optional
        Qualitative description of the district.

    holcColor : str, optional
        A string represent the color of the holcGrade of the district

    randomLat : float, optional
        A random latitude within the district (default is None).

    randomLong : float, optional
        A random longitude within the district (default is None).

    medIncome : int, optional
        Median household income for the district, to be filled later (default is None).

    censusTract : str, optional
        Census tract code for the district (default is None).


    Attributes
    ------------------------------
    self.coordinates
    self.holcGrade
    holcColor : str
        The color representation of the HOLC grade.
        • Districts with holc grade A should be assigned the color 'darkgreen'
        • Districts with holc grade B should be assigned the color 'cornflowerblue'
        • Districts with holc grade C should be assigned the color 'gold'
        • Districts with holc grade D should be assigned the color 'maroon'
        If there is no input for holcColor, it should be generated based on the holcGrade and the rule above.

    self.id
    self.description
    self.randomLat
    self.randomLong
    self.medIncome
    self.censusTract


    """

    def __init__(
        self,
        coordinates,
        holcGrade,
        id,
        description,
        holcColor=None,
        randomLat=None,
        randomLong=None,
        medIncome=None,
        censusTract=None,
        rank=None,
        percent=None,
    ):
        self.coordinates = coordinates
        self.holcGrade = holcGrade
        self.id = id
        self.description = description
        if holcColor:
            self.holcColor = holcColor
        else:
            if holcGrade == "A":
                self.holcColor = "darkgreen"
            elif holcGrade == "B":
                self.holcColor = "cornflowerblue"
            elif holcGrade == "C":
                self.holcColor = "gold"
            elif holcGrade == "D":
                self.holcColor = "maroon"
        self.randomLat = randomLat
        self.randomLong = randomLong
        self.medIncome = medIncome
        self.censusTract = censusTract
        self.rank = rank
        self.percent = percent


class RedLines:
    """
    A class to manage and analyze redlining district data.

    Attributes
    ----------
    districts : list of DetroitDistrict
        A list to store instances of DetroitDistrict.

    """

    def __init__(self, cacheFile=None):
        """
        Initializes the RedLines class without any districts.
        assign districts attribute to an empty list
        """
        self.districts = []
        if cacheFile:
            self.loadCache(cacheFile)

    def createDistricts(self, fileName):
        """
        Creates DetroitDistrict instances from redlining data in a specified file.
        Based on the understanding in step 1, load the file,parse the json object,
        and create 238 districts instance.
        Finally, store districts instance in a list,
        and assign the list to be districts attribute of RedLines.

        Parameters
        ----------
        fileName : str
            The name of the file containing redlining data in JSON format.

        Hint
        ----------
        The data for description attribute could be from
        one of the dict key with only number.

        """
        f = open(fileName)
        data = json.load(f)
        f.close()
        features = data["features"]
        for feature in features:
            coordinates = feature["geometry"].get("coordinates")[0][0]
            holcGrade = feature["properties"].get("holc_grade")
            id = feature["properties"].get("holc_id")
            description = feature["properties"].get("area_description_data").get("8")
            holcColor = feature["properties"].get("holc_color")
            district = DetroitDistrict(coordinates, holcGrade, id, description, holcColor)
            self.districts.append(district)

    def plotDistricts(self):
        """
        Plots the districts using matplotlib, displaying each district's location and color.
        Name it redlines_graph.png and save it to the current directory.
        """
        fig, ax = plt.subplots()
        for district in self.districts:
            ax.add_patch(plt.Polygon(district.coordinates, color=district.holcColor, alpha=0.5))
            ax.autoscale()
        plt.rcParams["figure.figsize"] = (15, 15)
        plt.savefig("redlines_graph.png")

    def generateRandPoint(self):
        """
        Generates a random point within the boundaries of each district.

        This method creates a mesh grid of points covering the geographical area of interest
        and then selects a random point within the boundary of each district.

        Attributes
        ----------
        self.districts : list of DetroitDistrict
            The list of district instances in the RedLines class.

        Note
        ----
        The random point is assigned as the randomLat and randomLong  for each district.
        This method assumes the 'self.districts' attribute has been populated with DetroitDistrict instances.

        """
        xgrid = np.arange(-83.5, -82.8, 0.004)
        ygrid = np.arange(42.1, 42.6, 0.004)
        xmesh, ymesh = np.meshgrid(
            xgrid, ygrid
        )  # create a mesh grid of points to make sure x and y coordinates are paired
        points = np.vstack(
            (xmesh.flatten(), ymesh.flatten())
        ).T  # flatten the grid and stack the x and y coordinates
        for district in self.districts:
            p = Path(district.coordinates)
            grid = p.contains_points(points)  # mask the points that are within the district
            point = points[
                random.choice(list(np.where(grid)[0]))
            ]  # randomly select a point within the district
            #print(f"{district} : {point}")
            district.randomLat = point[1]
            district.randomLong = point[0]

    def fetchCensus(self):
        """
        Fetches the census tract for each district in the list of districts using the FCC API.
        """

        api = "https://geo.fcc.gov/api/census/area"
        for district in tqdm(self.districts, desc="Fetching Census"):
            lat = district.randomLat
            lon = district.randomLong
            params = {"lat": lat, "lon": lon, "censusYear": "2010", "format": "json"}
            response = requests.get(api, params)
            census = []
            while True:
                if response.status_code == 200:
                    data = response.json()
                    district.censusTract = data["results"][0]["block_fips"][:11]
                    census.append(data)
                    break
                else:
                    response = requests.get(api, params)
    def fetch(self, params, colname = None):
            """
            Fetches data from the U.S. Census Bureau's API.

            This method sends a GET request to the specified API endpoint with the provided parameters.
            If the request is successful, the method returns the JSON response. If the request fails,
            the method retries the request until it is successful.

            Parameters
            ----------
            params : dict
                A dictionary containing the parameters to be sent with the request.

            Returns
            -------
            dict
                A dictionary containing the JSON response from the API.

            """
            api = "https://api.census.gov/data/2018/acs/acs5"
            key = "bff2ebd794e77fac281f258fe892bd1e36fe481b"
            params["key"]=key
            #print(params)
            while True:
                response = requests.get(api, params)
                if response.status_code == 200:
                    response_data = response.json()
                    break
                else:
                    print(response.status_code)
                    time.sleep(5)
            
            data = pd.DataFrame(response_data[1:], columns=response_data[0])
            data['tract_id'] = data['state'] + data['county'] + data['tract']
            if colname:
                data.rename(columns={data.columns[0]: colname}, inplace=True)
            return data

    def fetchIncome(self):
        """
        Retrieves the median household income for each district based on the census tract.

        This method requests income data from the ACS 5-Year Data via the U.S. Census Bureau's API
        for the year 2018. It then maps these incomes to the corresponding census tracts and updates
        the median income attribute of each district in `self.districts`.

        Note
        ----
        The method assumes that the `censusTract` attribute for each district is already set. It updates
        the `medIncome` attribute of each district based on the fetched income data. If the income data
        is not available or is negative, the median income is set to 0.

        """
        params = {"get": "B06011_001E", "for": "tract:*", "in": "state:26"}
        data = self.fetch(params, colname='medIncome')
        for district in self.districts:
            tract = district.censusTract
            try:
                income = int(data.loc[data['tract_id'] == tract, 'medIncome'])
            except:
                income = 0
            if income <= 0:
                district.medIncome = 0
            else:
                district.medIncome = income
            #print(f"{district.id} : {district.medIncome}")

    def cacheData(self, fileName):
        """
        Saves the current state of district data to a file in JSON format.
        Using the __dict__ magic method on each district instance, and save the
        result of it to a list.
        After creating the list, dump it to a json file with the inputted name.
        You should name the cache file as redlines_cache.json

        Parameters
        ----------
        filename : str
            The name of the file where the district data will be saved.
        """
        districts_data = [district.__dict__ for district in self.districts]
        with open(fileName, 'w') as f:
            json.dump(districts_data, f)


    def loadCache(self, fileName):
        """
        Loads district data from a cache JSON file if it exists.

        Parameters
        ----------
        fileName : str
            The name of the file from which to load the district data.
            You should name the cache file as redlines_cache.json

        Returns
        -------
        bool
            True if the data was successfully loaded, False otherwise.
        """
        if os.path.exists(fileName):
            with open(fileName, 'r') as f:
                data = json.load(f)
            for district_data in data:
                district = DetroitDistrict(**district_data)
                self.districts.append(district)
            return True
        else:
            return False

    def calcIncomeStats(self):
        """
        Use np.median and np.mean to
        Calculates the mean and median of median household incomes for each district grade (A, B, C, D).

        This method computes the mean and median incomes for districts grouped by their HOLC grades.
        The results are stored in a list following the pattern: [AMean, AMedian, BMean, BMedian, ...].
        After your calculations, you need to round the result to the closest whole int.
        Relate reading https://www.w3schools.com/python/ref_func_round.asp


        Returns
        -------
        list
            A list containing mean and median income values for each district grade in the order A, B, C, D.
        """
        grades = ['A', 'B', 'C', 'D']
        results = []
        for grade in grades:
            income = [district.medIncome for district in self.districts if district.holcGrade == grade]
            if income:
                mean = round(np.mean(income))
                median = round(np.median(income))
                results.extend([mean, median])
        return results


    def findCommonWords(self):
        """
        Analyzes the qualitative descriptions of each district category (A, B, C, D) and identifies the
        10 most common words unique to each category.

        This method aggregates the qualitative descriptions for each district category, splits them into
        words, and computes the frequency of each word. It then identifies and returns the 10 most
        common words that are unique to each category, excluding common English filler words.

        Returns
        -------
        list of lists
            A list containing four lists, each list containing the 10 most common words for each
            district category (A, B, C, D). The first list should represent grade A, and second for grade B,etc.
            The words should be in the order of their frequency.

        Notes
        -----
        - Common English filler words such as 'the', 'of', 'and', etc., are excluded from the analysis.
        - The method ensures that the common words are unique across the categories, i.e., no word
        appears in more than one category's top 10 list.
        - Regular expressions could be used for word splitting to accurately capture words from the text.
        - Counter from collections could also be used.

        """
        # List of filter, stop words, punctuation, and numbers
        filter_words = set(["the", "of", "and", "in", "to", "a", "is", "for", "on", "that"])
        stop_words = set(stopwords.words('english'))
        punctuations = string.punctuation

        # Combine two lists
        filter_words = filter_words.union(stop_words)

        # Store the words for each grade
        words = {grade:[] for grade in ['A', 'B', 'C', 'D']}
        for district in self.districts:
            # Remove filter words, punctuation, numbers, and make all words lowercase
            description_text = re.sub(r'\d+', '', district.description.lower())
            description_text = re.sub(r'[^\w\s]', '', description_text)
            description = [word for word in description_text.split() if word not in filter_words]
            words[district.holcGrade].append(description)

        # count the words for each grade
        counts = {grade: Counter() for grade in ['A', 'B', 'C', 'D']}
        for grade in ['A', 'B', 'C', 'D']:
            for description in words[grade]:
                counts[grade].update(description) # iterate through the list of description and count the words
         
        # Find the 10 most common words for each grade and make sure they are unique across different lists
        common_words = {grade:[] for grade in ['A', 'B', 'C', 'D']}
        used_words = set()
        for grade in ['A', 'B', 'C', 'D']:
            for word, count in counts[grade].most_common():
                if len(common_words[grade]) == 10:
                    break
                if word in used_words:
                    continue
                else:
                    common_words[grade].append(word)
                    used_words.add(word)
        
        # change the dict to list of lists
        result = [common_words[grade] for grade in ['A', 'B', 'C', 'D']]
        # print(result)
        return result


    def calcRank(self):
        """
        Calculates and assigns a rank to each district based on median income.

        This method sorts the districts in descending order of their median income and then assigns
        a rank to each district, with 1 being the highest income district.

        Note
        ----
        The rank is assigned based on the position in the sorted list, so the district with the highest
        median income gets a rank of 1, the second-highest gets 2, and so on. Ties are not accounted for;
        each district will receive a unique rank.

        Important:
        If you do the extra credit, you need to edit the __init__ of DetroitDistrict adding another arg "rank" with
        default value to be None. Not doing so might cause the load cache method to fail if you use the ** operator in load cache.

        Attribute
        ----
        rank

        """
        sort_districts = sorted(self.districts, key=lambda x: x.medIncome, reverse=True)
        for rank, district in enumerate(sort_districts, 1):
            district.rank = rank
        
        
    
    def calcPopu(self):
        """
        Fetches and calculates the percentage of Black or African American residents in each district.

        This method fetch the total and Black populations for each census tract in Michigan from
        the U.S. Census Bureau's API, like the median income data.  It then calculates the percentage of Black residents in each tract
        and assigns this value to the corresponding district percent attribute.

        Note
        ----
        The method assumes that the census tract IDs in the district data match those used by the Census Bureau.
        The percentage is rounded to two decimal places. If the Black population is zero, the percentage is set to 0.
        Elif the total population is zero, the percentage is set to 1.

        Important:
        If you do the extra credit, you need to edit the __init__ of DetroitDistrict adding another arg "percent" with
        default value to be None. Not doing so might cause the load cache method to fail if you use the ** operator in load cache.
        https://api.census.gov/data/2018/acs/acs5?get=B02001_001E&for=tract:26163550700&in=state:26&key=bff2ebd794e77fac281f258fe892bd1e36fe481b


        Attribute
        ----
        percent

        """

        def _calcuate_percent(row):
            print(row['tract_id'], row['afam_pop'], row['total_pop'])
            if int(row['afam_pop']) == 0:
                return 0
            elif int(row['total_pop']) == 0:
                return 1
            else:
                return round((int(row['afam_pop']) / int(row['total_pop'])), 2)

        params = {"get": "B02001_003E", "for": f"tract:*", "in": f"state:26"}
        afam_data = self.fetch(params, colname='afam_pop')
        params["get"] = "B01001_001E"
        total_data = self.fetch(params, colname='total_pop')
        pop_data = pd.merge(afam_data, total_data, on='tract_id')
        pop_data['percent'] = pop_data.apply(_calcuate_percent, axis=1)

        for district in self.districts:
            district.percent = pop_data.loc[pop_data['tract_id'] == district.censusTract, 'percent']
        
            
    def comment(self):
        """
        Look at the
        districts in each category, A, B, C and D. Are there any trends that you see? Share 1 paragraph of your
        findings. And a few sentences(more than 50 words) about how this exercise did or did not change your understanding of
        residential segregation. Print you thought in the method.
        """
        print("")

    def sample(self):
        """
        This is a sample method that you can use to test your implementation.
        """
        # randomly selected 5 districts
        sample = random.sample(self.districts, 5)
        for district in sample:
            print(district.id, district.holcGrade, district.medIncome, district.censusTract, district.rank, district.percent)


# Use main function to test your class implementations.
# Feel free to modify the example main function.
def main():
    myRedLines = RedLines("redlines_cache.json")
    income = myRedLines.calcIncomeStats()
    income = [(income[i], income[i+1]) for i in range(0, len(income), 2)]
    grade = ['A', 'B', 'C', 'D']
    for i in range(4):
        print(f"{grade[i]} Mean: {income[i][0]}, Median: {income[i][1]}")
    data_percent = [[district.id, district.holcGrade, district.medIncome, district.percent] for district in myRedLines.districts]
    df_percent = pd.DataFrame(data_percent, columns=['id', 'grade', 'medincome', 'percent'])
    print(df_percent.groupby('grade')[['medincome', 'percent']].mean())



if __name__ == "__main__":
    main()


A Mean: 98331, Median: 113220
B Mean: 65858, Median: 70125
C Mean: 42635, Median: 34194
D Mean: 30118, Median: 25463
          medincome   percent
grade                        
A      98330.600000  0.372667
B      65857.948718  0.390513
C      42635.319672  0.522869
D      30117.645161  0.593871


In [ ]:
district.